In [1]:
!pip install tensorflow==2.20.0
!pip install numpy opencv-python scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import cv2
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import Sequence

In [3]:
# =====================================================
# 1. Video Loader
# =====================================================
def read_video_frames(video_path, seq_len=24):
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total < seq_len:
        return None

    idxs = np.linspace(0, total - 1, seq_len).astype(int)
    frames = []

    for i in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            return None
        frame = cv2.resize(frame, (224, 224))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()
    return np.array(frames)


class VideoLoader(Sequence):
    def __init__(self, X, y, seq_len=24, batch_size=4, **kwargs):
        super().__init__(**kwargs)
        self.X = X
        self.y = y
        self.seq = seq_len
        self.batch = batch_size

    def __len__(self):
        return len(self.X) // self.batch

    def __getitem__(self, idx):
        batch_files = self.X[idx * self.batch:(idx + 1) * self.batch]
        batch_labels = self.y[idx * self.batch:(idx + 1) * self.batch]

        videos = []
        labels = []

        for f, lbl in zip(batch_files, batch_labels):
            frames = read_video_frames(f, self.seq)
            if frames is not None:
                videos.append(frames)
                labels.append(lbl)

        return np.array(videos) / 255.0, np.array(labels)


In [4]:
# =====================================================
# 2. Load Dataset
# =====================================================
def load_dataset(root="D:/FINAL_YEAR_PROJECTS/Deep_face/data"):
    X, y = [], []

    for label, folder in enumerate(["fake", "real"]):

        path = os.path.join(root, folder, "*.mp4")
        files = sorted(glob.glob(path))

        print(f"[INFO] {folder.upper()}: {len(files)} videos")

        X.extend(files)
        y.extend([label] * len(files))

    return X, y


In [5]:
# =====================================================
# 3. Build Model
# =====================================================
def build_model(seq_len=24, height=224, width=224, channels=3):

    input_tensor = Input(shape=(seq_len, height, width, channels))

    base_model = EfficientNetB0(
        include_top=False,
        weights="imagenet",
        pooling="avg",
        input_shape=(height, width, channels)
    )
    base_model.trainable = False

    x = TimeDistributed(base_model)(input_tensor)
    x = LSTM(512, return_sequences=False)(x)
    output = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=input_tensor, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    return model


In [6]:
# =====================================================
# 4. Train Function
# =====================================================
def train():

    SEQ_LEN = 24
    HEIGHT = 224
    WIDTH = 224
    CHANNELS = 3

    X, y = load_dataset()

    if len(X) == 0:
        raise ValueError("❌ Dataset is empty. Check your paths!")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=True
    )

    train_loader = VideoLoader(X_train, y_train, seq_len=SEQ_LEN, batch_size=2)
    test_loader = VideoLoader(X_test, y_test, seq_len=SEQ_LEN, batch_size=2)

    model = build_model(
        seq_len=SEQ_LEN,
        height=HEIGHT,
        width=WIDTH,
        channels=CHANNELS
    )

    print("\n[INFO] Training started...\n")

    model.fit(
        train_loader,
        epochs=15,
        validation_data=test_loader,
        callbacks=[
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2),
            tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
        ]
    )

    # =============================
    # ✔ Correct Saving Formats
    # =============================
    MODEL_DIR = "D:/FINAL_YEAR_PROJECTS/Deep_face/model_real"
    os.makedirs(MODEL_DIR, exist_ok=True)

    model.save(f"{MODEL_DIR}/model.keras")                # Recommended format
    model.save(f"{MODEL_DIR}/model.h5")                   # Legacy format
    model.save_weights(f"{MODEL_DIR}/model.weights.h5")   # Correct weights name

    print("\n✅ [INFO] Model saved successfully!")
    print("Final Train Acc:", model.history.history["accuracy"][-1], "| Final Val Acc:", model.history.history["val_accuracy"][-1])



In [7]:
# pip uninstall keras -y

In [8]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tensorflow in c:\users\dell\appdata\local\programs\python\python310\lib\site-packages (2.20.0)




[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install tensorflow==2.20.0


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, Input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import Sequence

In [11]:
# ----------------------------
# Start Training
# ----------------------------
train()

[INFO] FAKE: 5645 videos
[INFO] REAL: 890 videos

[INFO] Training started...

Epoch 1/15
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 2212s 800ms/step - accuracy: 0.8651 - loss: 0.4046 - val_accuracy: 0.8576 - val_loss: 0.4094 - learning_rate: 1.0000e-04
Epoch 2/15
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 2408s 921ms/step - accuracy: 0.8655 - loss: 0.3970 - val_accuracy: 0.8576 - val_loss: 0.4102 - learning_rate: 1.0000e-04
Epoch 3/15
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 2474s 947ms/step - accuracy: 0.8655 - loss: 0.3957 - val_accuracy: 0.8576 - val_loss: 0.4094 - learning_rate: 1.0000e-04
Epoch 4/15
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 3088s 1s/step - accuracy: 0.8655 - loss: 0.3955 - val_accuracy: 0.8576 - val_loss: 0.4093 - learning_rate: 5.0000e-05
Epoch 5/15
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 2838s 1s/step - accuracy: 0.8655 - loss: 0.3958 - val_accuracy: 0.8576 - val_loss: 0.4105 - learning_rate: 5.0000e-05
Epoch 6/15
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 3142s 1s/step - accuracy: 0.8655 - loss: 0.3954 - val_accuracy: 0.8576 


✅ [INFO] Model saved successfully!
Final Train Acc: 0.8655060529708862 | Final Val Acc: 0.8575804233551025


# TEST

In [ ]:
import tensorflow as tf
import numpy as np
import cv2

# ----------------------------
# Load model
# ----------------------------
model = tf.keras.models.load_model(
    "D:/FINAL_YEAR_PROJECTS/Deep_face/model_real/model.keras"
)

SEQ_LEN = 24
IMG_SIZE = 224

# ----------------------------
# Video reader (SAFE)
# ----------------------------
def read_video_for_test(video_path, seq_len=24):
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total <= 0:
        cap.release()
        raise ValueError("❌ Unable to read video")

    frames = []

    idxs = np.linspace(0, total - 1, min(seq_len, total)).astype(int)

    for i in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()

    # 🔁 PAD if video is short
    while len(frames) < seq_len:
        frames.append(frames[-1])

    frames = np.array(frames, dtype=np.float32) / 255.0
    return frames


# ----------------------------
# Predict function
# ----------------------------
def predict_video(video_path):
    frames = read_video_for_test(video_path, SEQ_LEN)
    frames = np.expand_dims(frames, axis=0)  # (1, 24, 224, 224, 3)

    prediction = model.predict(frames, verbose=0)[0][0]

    print(f"\n🔍 Confidence Score: {prediction:.4f}")

    if prediction > 0.5:
        print("🔵 RESULT: REAL VIDEO")
    else:
        print("🔴 RESULT: FAKE VIDEO")


# ----------------------------
# Test
# ----------------------------
video_path = r"D:/FINAL_YEAR_PROJECTS/DEEP_FACE/data/fake/id0_id2_0008.mp4"
predict_video(video_path)



🔍 Confidence Score: 0.1434
🔴 RESULT: FAKE VIDEO
